In [ ]:
!pip install torch torchvision transformers datasets accelerate peft bitsandbytes

In [ ]:
import torch
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration,Trainer, TrainingArguments
from datasets import load_dataset
from PIL import Image
import requests
from torch.utils.data import DataLoader
from io import BytesIO
from torch.utils.data import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [ ]:

# Load the pre-trained model in 8-bit quantized mode
model_name = "Qwen/Qwen2.5-VL-3B-Instruct"

# Load the model with 8-bit quantization
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Use float16 for better performance
    device_map="auto",  # Automatically map layers to the GPU
    load_in_8bit=True,  # Enable 8-bit quantization
)

# Prepare the model for k-bit training
model = prepare_model_for_kbit_training(model)

# Load the processor
processor = AutoProcessor.from_pretrained(model_name)

In [ ]:

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,  # Rank of the low-rank matrices
    lora_alpha=32,  # Scaling factor for LoRA updates
    target_modules=[
        "q_proj",  # Query projection
        "k_proj",  # Key projection
        "v_proj",  # Value projection
    ],
    lora_dropout=0.05,  # Dropout probability for LoRA layers
    bias="none",  # Do not train biases
    task_type="CAUSAL_LM",  # Task type: Causal Language Modeling
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

In [ ]:
dataset = load_dataset("nlphuji/flickr30k")
dataset

In [ ]:
# Function to filter and subset the dataset by split
def filter_and_subset(single_dataset, split_name, subset_size=1000):
    # Filter by split
    filtered_dataset = single_dataset.filter(lambda example: example["split"] == split_name)

    # Select the first subset_size samples
    return  filtered_dataset.shuffle(seed=42)

# Subset the dataset for train, validation, and test splits
train_dataset_subset = filter_and_subset(dataset, "train", subset_size=1000)
val_dataset_subset = filter_and_subset(dataset, "val", subset_size=1000)
test_dataset_subset = filter_and_subset(dataset, "test", subset_size=1000)

In [ ]:
print(train_dataset_subset)
print(val_dataset_subset)
print(test_dataset_subset)

In [ ]:
import random

dataset_large = train_dataset_subset["test"]  # 29000 samples
dataset_medium = val_dataset_subset["test"]  # 1014 samples
dataset_small = test_dataset_subset["test"]  # 1000 samples

random_indices_large = random.sample(range(len(dataset_large)), 1000)
random_indices_medium = random.sample(range(len(dataset_medium)), 150)
random_indices_small = random.sample(range(len(dataset_small)), 150)

train = dataset_large.select(random_indices_large)
test = dataset_medium.select(random_indices_medium)
val = dataset_small.select(random_indices_small)

In [ ]:
train
test
val

In [ ]:
train

In [ ]:
# del dataset
del train_dataset_subset
del val_dataset_subset
del test_dataset_subset

In [ ]:
# Step 3: Preprocess Data - Randomly sample one caption per image
def preprocess_data(example):
    caption = random.choice(example["caption"])  # Randomly sample one caption
    return {"image": example["image"], "caption": caption}

# Apply preprocessing to all subsets
train_dataset_processed = train.map(preprocess_data, batched=False)
val_dataset_processed = test.map(preprocess_data, batched=False)
test_dataset_processed = val.map(preprocess_data, batched=False)


In [ ]:
# Apply preprocessing to all subsets
print("Processing train dataset...")
train_dataset_processed = train.map(preprocess_data, batched=False, remove_columns=train.column_names)
print("Processing validation dataset...")
val_dataset_processed = test.map(preprocess_data, batched=False, remove_columns=test.column_names)
print("Processing test dataset...")
test_dataset_processed = val.map(preprocess_data, batched=False, remove_columns=val.column_names)

In [ ]:
# Check the structure of the dataset to understand what "image" contains
sample = train_dataset_processed[0]
print(sample)
print(f"Sample image type: {type(sample['image'])}")
print(f"Sample caption: {sample['caption']}")

In [ ]:
# Custom data collator function with proper image handling
def collate_fn(batch):
    images = []
    captions = []

    for example in batch:
        image_path_or_obj = example["image"]
        caption = example["caption"]

        # Load image - handle different possible types
        try:
            # If already a PIL Image
            if isinstance(image_path_or_obj, Image.Image):
                image = image_path_or_obj.convert("RGB")
            # If it's a string path or URL
            elif isinstance(image_path_or_obj, str):
                if image_path_or_obj.startswith("http"):
                    response = requests.get(image_path_or_obj)
                    image = Image.open(BytesIO(response.content)).convert("RGB")
                else:
                    image = Image.open(image_path_or_obj).convert("RGB")
            else:
                raise TypeError(f"Unsupported image type: {type(image_path_or_obj)}")

            images.append(image)
            captions.append(caption)
        except Exception as e:
            print(f"Error processing image: {e}")
            continue  # Skip this example if there's an error

    if not images:  # Handle case where all images failed to load
        raise ValueError("No valid images found in batch")

    # Process inputs
    inputs = processor(
        text=captions,
        images=images,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        add_special_tokens=True,
    )
    # Labels are the same as input_ids for language modeling
    inputs["labels"] = inputs["input_ids"].clone()

    return inputs

In [ ]:
sample = collate_fn([train_dataset_processed[0]])
print(sample.keys())
print(sample["input_ids"].shape)
print(sample["attention_mask"].shape)
print(sample["labels"].shape)

In [ ]:
# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,  # Small batch size to fit in memory
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,  # Simulate larger batch sizes
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    remove_unused_columns=False,
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=5e-5,
    fp16=True,  # Enable mixed precision training
    save_total_limit=2,  # Save only the best two checkpoints
    report_to="none",
)

In [ ]:
# Create the trainer with the data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_processed,
    eval_dataset=val_dataset_processed,
    data_collator=collate_fn,
)

In [ ]:
# Fine-Tune the Model
trainer.train()

In [ ]:

# Evaluate the Model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

In [ ]:
# Save the fine-tuned model
os.makedirs("./fine_tuned_qwen2.5_vl_lora", exist_ok=True)
model.save_pretrained("./fine_tuned_qwen2.5_vl_lora")
processor.save_pretrained("./fine_tuned_qwen2.5_vl_lora")